# Install required libraries

In [ ]:
!pip install opencv-python torch torchvision

In [ ]:
import cv2
import os

def extract_frames(video_source, output_folder, save_every_n_frames=5):
    """
    Extracts frames from a video and saves every nth frame to the specified folder.

    Args:
        video_source (str): Path to the video file.
        output_folder (str): Directory to save the extracted frames.
        save_every_n_frames (int): Save every nth frame (default is 5).
    """
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_source)

    if not cap.isOpened():
        print(f"Error opening video source: {video_source}")
        return

    # Process video frames
    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save every nth frame
        if frame_number % save_every_n_frames == 0:
            frame_filename = os.path.join(output_folder, f'frame_{frame_number}.jpg')
            cv2.imwrite(frame_filename, frame)
            print(f"Saved: {frame_filename}")

        frame_number += 1

    cap.release()

    print("Frame extraction complete.")

# Example usage
video_source = '/content/1643-148614430_small.mp4'
output_folder = '/content/1643_extracted_frames-new'
extract_frames(video_source, output_folder, save_every_n_frames=5)

# Use MTCNN with YOLOv5(underlying model) for face detection in frames

In [ ]:
!pip install facenet_pytorch

In [ ]:
import cv2
import torch
from facenet_pytorch import MTCNN
from PIL import Image
import os

# Initialize the MTCNN model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)

# Directory to save the extracted frames
output_folder = '/content/1643_extracted_frames_highlighted_faces'
os.makedirs(output_folder, exist_ok=True)


def process_frame(frame, frame_number):
    try:
        # Convert OpenCV image (BGR) to PIL image (RGB)
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Detect faces
        boxes, probs = mtcnn.detect(pil_image)

        if boxes is not None:
            for box in boxes:

                x1, y1, x2, y2 = map(int, box)
                width = x2 - x1
                height = y2 - y1


                top_margin = int(height * 0.5)
                side_margin = int(width * 0.3)

                # Modify bounding box
                x1 = max(0, x1 - side_margin)
                y1 = max(0, y1 - top_margin)
                x2 = min(frame.shape[1], x2 + side_margin)
                y2 = min(frame.shape[0], y2 + side_margin)

                # Draw bounding boxes on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        return frame
    except Exception as e:
        print(f"Error processing frame: {e}")
        return frame

# Open video file
video_source = '/content/1643-148614430_small.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_source)

# Check if video source is opened successfully
if not cap.isOpened():
    print(f"Error opening video source: {video_source}")
    exit()

# Process video frames
frame_number = 0
save_every_n_frames = 5  # Save every 5th frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame and highlight faces
    frame = process_frame(frame, frame_number)

    # Save every 5th frame with highlighted faces
    if frame_number % save_every_n_frames == 0:
        frame_filename = os.path.join(output_folder, f'frame_{frame_number}.jpg')
        cv2.imwrite(frame_filename, frame)
        print(f"Saved: {frame_filename}")

    frame_number += 1

# Release resources
cap.release()

print("Frame extraction complete.")

In [ ]:
# frame extraction no detection
import cv2
import os

# Directory to save the extracted frames
output_folder = '/content/1643_extracted_frames-new'
os.makedirs(output_folder, exist_ok=True)

# Open video file
video_source = '/content/1643-148614430_small.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_source)

# Check if video source is opened successfully
if not cap.isOpened():
    print(f"Error opening video source: {video_source}")
    exit()

# Process video frames
frame_number = 0
save_every_n_frames = 5  # Save every 5th frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Save every 5th frame
    if frame_number % save_every_n_frames == 0:
        frame_filename = os.path.join(output_folder, f'frame_{frame_number}.jpg')
        cv2.imwrite(frame_filename, frame)
        print(f"Saved: {frame_filename}")

    frame_number += 1

# Release resources
cap.release()

print("Frame extraction complete.")


# Using GFPGAN for face enhancement

In [ ]:
!git clone https://github.com/TencentARC/GFPGAN.git

In [ ]:
!pip install basicsr facexlib

In [ ]:
!pip install -r /content/GFPGAN/requirements.txt

In [ ]:
# Change to the GFPGAN directory
%cd /content/GFPGAN

# List files to confirm VERSION file is present
!ls

# Run the setup.py script
!python setup.py develop
!pip install --use-pep517
!pip install -r requirements.txt

In [ ]:
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

In [ ]:
!python /content/GFPGAN/inference_gfpgan.py -i /content/1643_extracted_frames-new -o /content/enhanced_faces -v 1.3 -s 2

# Install the CodeFormer model for face Enhancement
Link: [CodeFormer](https://shangchenzhou.com/projects/CodeFormer/)

In [ ]:
!git clone https://github.com/sczhou/CodeFormer

In [ ]:
!pip install basicsr facexlib

### run the next cell only if error related to torch occurs




In [ ]:
!pip uninstall torch torchvision -y
!pip install torch==1.13.0 torchvision==0.14.0 -f https://download.pytorch.org/whl/cu121/torch_stable.html

In [ ]:
!pip install -r /content/CodeFormer/requirements.txt

In [ ]:
%cd /content/CodeFormer

!python basicsr/setup.py develop

In [ ]:
!pip install ninja ffmpeg dlib

In [ ]:
%cd /content/CodeFormer
!python scripts/download_pretrained_models.py CodeFormer

In [ ]:
%cd /content/CodeFormer

!python inference_codeformer.py -w 0.5 --input_path /content/1643_extracted_frames-new --output_path /content/enhanced_faces